In [6]:
from os.path import join
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode

In [7]:
class Regression(nn.Module):
    def __init__(self):
        super(Regression, self).__init__()
        self.c1 = nn.Conv2d(1, 4, kernel_size=3)
        self.p1 = nn.MaxPool2d(2)
        
        self.c2 = nn.Conv2d(4, 8, kernel_size=3)
        self.p2 = nn.MaxPool2d(2)
        self.d1 = nn.Dropout2d()
        
        self.c3 = nn.Conv2d(8, 16, kernel_size=3)
        self.p3 = nn.MaxPool2d(2)
        self.d2 = nn.Dropout2d()
        
        self.f1 = nn.Flatten()
        self.l1 = nn.Linear(45 * 50 * 16, 64)
        self.d3 = nn.Dropout()
        
        self.l2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.p1(F.relu(self.c1(x)))
        x = self.d1(self.p2(F.relu(self.c2(x))))
        x = self.d2(self.p3(F.relu(self.c3(x))))
        x = self.d3(F.relu(self.l1(self.f1(x))))
        x = F.relu(self.l2(x))
        return x

    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Regression()
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

print(model, "on", device, "with", sum(p.numel() for p in model.parameters() if p.requires_grad), "parameters")

Regression(
  (c1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout2d(p=0.5, inplace=False)
  (c3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): Dropout2d(p=0.5, inplace=False)
  (f1): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=36000, out_features=64, bias=True)
  (d3): Dropout(p=0.5, inplace=False)
  (l2): Linear(in_features=64, out_features=1, bias=True)
) on cuda:0 with 2305633 parameters


In [8]:
class ImageDataset(Dataset):
    def __init__(self, im_dir):
        df = pd.read_csv(join(im_dir, "label.csv"), sep=";")
        self.images = df["FileName"].apply(lambda im: join(im_dir, im))
        self.labels = df["Angle"]
        
    def __len__(self) -> int:
        return len(self.labels)
    
    def __getitem__(self, index) -> (torch.Tensor, float):
        image = read_image(self.images[index], ImageReadMode.GRAY) / 255
        label = self.labels[index]
        return image, torch.Tensor([label])

    
batch_size = 32

train_dataloader = DataLoader(ImageDataset("../data/train/"), batch_size=batch_size, shuffle=True)
test_dataloader  = DataLoader(ImageDataset("../data/test/"), batch_size=batch_size, shuffle=True)

In [9]:
def run_epoch(training, verbose=True):
    dataloader = None
    if training:
        dataloader = train_dataloader
        model.train()
    else:
        dataloader = test_dataloader
        model.eval()
    
    run_loss, run_error = 0, 0
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_pred = model(x_batch.to(device)).to("cpu")
        loss = criterion(y_pred, y_batch)
        if training:
            loss.backward()
            optimizer.step()
        
        run_loss += loss.item()
        run_error += (y_pred - y_batch).abs().sum().item()
        
    if verbose:
        print("Training Loss: {:10.3f} | MAE: {:7.3f}".format(run_loss / len(dataloader), run_error / len(dataloader.dataset)))
    return run_error / len(dataloader.dataset)

In [10]:
n_epochs = 15
for n in range(n_epochs):
    print("Epoch {:4d}".format(n), end=" | ")
    run_epoch(training=True)
print("Test", end = " | ")
run_epoch(training=False)

Epoch    0 | Training Loss:  10633.257 | MAE:  86.261
Epoch    1 | Training Loss:   5116.094 | MAE:  58.574
Epoch    2 | Training Loss:   4264.658 | MAE:  53.448
Epoch    3 | Training Loss:   4151.994 | MAE:  52.283
Epoch    4 | Training Loss:   3792.524 | MAE:  50.218
Epoch    5 | Training Loss:   4103.812 | MAE:  52.267
Epoch    6 | Training Loss:   3461.709 | MAE:  47.635
Epoch    7 | Training Loss:   3561.758 | MAE:  48.562
Epoch    8 | Training Loss:   3438.703 | MAE:  47.471
Epoch    9 | Training Loss:   3544.240 | MAE:  48.316
Epoch   10 | Training Loss:   3330.946 | MAE:  46.452
Epoch   11 | Training Loss:   3251.648 | MAE:  46.469
Epoch   12 | Training Loss:   3468.898 | MAE:  47.843
Epoch   13 | Training Loss:   3592.100 | MAE:  48.594
Epoch   14 | Training Loss:   3253.206 | MAE:  45.299
Test | Training Loss:    956.167 | MAE:  28.900


28.900204477586165